# How-To Deploy a Web Application

```{note}
Fork this repository in to your own GitHub account to follow along. GitHub provides detailed documentation on different methods of forking that can be found via this [link to GitHub docs](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/working-with-forks/fork-a-repo?tool=webui).
``` 

## Run the application locally

Let's start by running the application locally. Python and Flask are required. If you do not already have python installed on your machine you can do so by following the instructions found at this [link to Python Downloads](https://www.python.org/downloads/).

With python installed running the following will install Flask

```{note}
The ! is used so the Jupyter notebook cells can be run directly. This is not required if you want to run the code on the command line yourself. 
```

In [ ]:
!pip install -r flask-app/requirements.txt

Flask is now installed and the application can be launched with the following command

In [4]:
%cd flask-app
!flask run

[Errno 2] No such file or directory: 'flask-app'
/home/ncote/Code/docker-how-to/flask-app
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
^C


UsageError: Line magic function `%` not found.


The link above should direct you to a local website. If not you can use the following link in your browser : [http://127.0.0.1:5000](http://127.0.0.1:5000)

Use `CTRL+C` to exit the application or stop the running cell if run directly in the jupyter notebook. 

## Create a Container File

```{note}
Docker or Podman are required on your local machine for the next steps. Installation instructions for your OS can be found at this [link to Podman Installation Instructions](https://podman.io/docs/installation) or this [link to Docker Installation Instructions](https://docs.docker.com/engine/install/). If you do not want to build the image locally, using GitHub Actions to build an image is covered below. 

With either Docker or Podman builds the typical set of instructions on how to build a container is called a Dockerfile. 

There's a number of different base container images available for various different languages, operating systems, and applications. Since the example application used in this how-to is based on python an official python container image is used as the base. Then files required for the application are copied in. Software dependencies are installed. The port the application runs on is exposed so it can be accessed outside the container. Lastly the command to run the application is supplied and will only be run when the container is started. 

Create a file named `Dockerfile` that exists in the root of the repository. An example `Dockerfile`, with comments, for the flask-app application can be found below.

```
# Use an official Python runtime as a base image
FROM python:3.8-slim

# Copy all the filed in the flask-app directory in to the root directory in the container
COPY flask-app/ /

# Install the python requirements
RUN pip install --no-cache-dir -r requirements.txt

# Flask runs on port 5000 and it needs to be exposed outside the container
EXPOSE 5000

# The command to run the Flask application when the container starts
CMD ["python3", "./wsgi.py"]
```

The container image can be built locally now by running either of the following commands:

In [ ]:
%cd ..
!docker build -t app-test .

In [ ]:
!podman build -t app-test .

Once the image has been built it can be run with either of the following commands:

In [ ]:
!docker run -p 5000:5000 app-test

In [ ]:
!podman run -p 5000:5000 app-test

The example web application can be viewed at [http://127.0.0.1:5000](http://127.0.0.1:5000). 

## Create GitHub Workflow

### Setup directories

GitHub Actions and Workflows can be utilized to build the container image instead of building it locally. YAML files are utilized inside the `.github/workflows/` directory to define the jobs and actions the automated workflow should follow. First create a directory in the root of the repository named `.github`. Inside the `.github` folder create another folder named `workflows`. 

### Repository Secrets

GitHub provides a way to inject secret information, like login credentials, into workflow jobs. From the GitHub main repository website select the repository Settings from the far right of the navigation bar at the top of the page. On the left side table of contents there is a Security section with a dropdown for Secrets and variables. Expand the options and select Actions to land at the following page. 

![GitHub Repo Secrets Screenshot](images/github_secrets.png "GitHub Repo Secrets")

```{note}
You can also append `/settings/secrets/actions` to the repositories base URL. For example `https://github.com/NicholasCote/docker-how-to/settings/secrets/actions`
```

Select the New repository secret button to create and add a new secret. The secret is not readable after it is added. It can only be changed or deleted. The example above added `DOCKERHUB_USERNAME` and `DOCKERHUB_TOKEN` secrets to the repository that can be utilized to push images to a Docker Hub container registry. 

```{note}
In order to completely follow the example a Docker Hub account and an Access Token for that account is required. Access tokens for Docker Hub accounts can be created at the following [link to security settings](https://hub.docker.com/settings/security). Add the Hub username and Access Token to the repository by following the instructions above. 
```

### Create Docker build YAML file

Create a file named `docker-build.yaml` in the `.github/workflows/` directory. This file will define the different components of the automated workflow. First define the name of your workflow

```yaml
name: Build & push container
```

Next define what action triggers the workflow. In the example it will run when any pushes are made to the `flask-app/` directory in the main branch

```yaml
on:
  push:
    paths:
      - flask-app/**
    branches:
      - main
```

The last step is to define the job(s) that need to run as part of the workflow. The example only defines a single job, but multiple jobs can be configured in a workflow. GitHub provides different virtual machines, called runners, that can be used to execute jobs. The latest ubuntu image is used for the job in the example but other options are available and a full list can be found at this [link to available GitHub runners](https://docs.github.com/en/actions/using-github-hosted-runners/about-github-hosted-runners/about-github-hosted-runners#supported-runners-and-hardware-resources). The following start to the example defines a job named build-image that runs on ubuntu-latest.

```yaml
jobs:
  build-image:
    runs-on: ubuntu-latest
```

Each job consists of steps to run through. The job to build and push a docker image consists of 4 steps. First checkout the GitHub repo, login to Docker Hub with the repository secrets, get the current date and time to apply as the image tag, build, and push the image. 

```{note}
The date and time is used to provide a unique tag value. Using `:latest` as the tag is not best practice as it can not create a difference in Helm chart versions to trigger using a new image. 
```

```yaml
    steps:
      # Step 1 is to checkout the github repo used to build the Dockerfile
      - name: Check out the repo
        uses: actions/checkout@v3
      # Step 2 is to login to docker hub so the image can be pushed
      - name: Login to Docker Hub
        uses: docker/login-action@v2
        # GitHub repository secrets are used as variables to provide login information to Docker Hub
        with:
          username: ${{ secrets.DOCKERHUB_USERNAME }}
          password: ${{ secrets.DOCKERHUB_TOKEN  }}
      # Get the date and send to GITHUB_OUTPUT to apply as image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H')" >> $GITHUB_OUTPUT
      # Build and push the docker image
      - name: Build Docker image
        uses: docker/build-push-action@v4
        with:
          # Provide the current directory as build context 
          context: .
          # Specify where the Dockerfile is located in relation to the repo base path
          file: Dockerfile
          # Enable the push to docker hub
          push: true
          # Provide the tags to apply to the image, this example uses the latest image tag 
          tags: |
            ncote/docker-example:${{ steps.date.outputs.date }}
```


The last step uses an action that pushes the image created to Docker Hub. The only unique value that needs to be updated is the last line, the image name and tag. The part before the `/` is the name of the container registry. The part after the `/` is the image name and tag. The `${{ }}` is how variables are defined in the workflow file. The example uses the output from the 2nd step as a variable to use for the image tag. An example of how to create a workflow that also pushes the image to Docker Hub will be explored further down in the documentation. This workflow is a great way to test that changes made work before pushing images out or updating Helm charts. In full the example workflow looks like the following:

```yaml
name: Automation to build a container image

on:
  push:
    paths:
      - flask-app/**
    branches:
      - main

jobs:
  build-image:
    runs-on: ubuntu-latest
    steps:
      # Step 1 is to checkout the github repo used to build the Dockerfile
      - name: Check out the repo
        uses: actions/checkout@v3
      # Step 2 is to login to docker hub so the image can be pushed
      - name: Login to Docker Hub
        uses: docker/login-action@v2
        # GitHub repository secrets are used as variables to provide login information to Docker Hub
        with:
          username: ${{ secrets.DOCKERHUB_USERNAME }}
          password: ${{ secrets.DOCKERHUB_TOKEN  }}
      # Get the date and send to GITHUB_OUTPUT to apply as image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H')" >> $GITHUB_OUTPUT
      # Build and push the docker image
      - name: Build Docker image
        uses: docker/build-push-action@v4
        with:
          # Provide the current directory as build context 
          context: .
          # Specify where the Dockerfile is located in relation to the repo base path
          file: Dockerfile
          # Enable the push to docker hub
          push: true
          # Provide the tags to apply to the image, this example uses the latest image tag 
          tags: |
            ncote/docker-example:${{ steps.date.outputs.date }}
```

### Run the Workflow

Now that the workflow is in place push the changes to GitHub and get the new workflow defined for the repository. The trigger is any change to a file in the `flask-app/` directory. Navigate to that directory, open the templates folder, and edit `home.html`. Let's just add a second header to the HTML below `<h1>A Demo WebApp</h1>` with our name to test the build and container registry push. An example line of what the code looks like can be seen below: 

```html
        <h2>Nick Cote</h2>
```

With that in place commit and push the changes to the repository. Once the command has been run navigate back to the repository in a web browser. The 4th option from the left on the top navigation bar is for Actions. Click this to be directed to a list of all workflow runs. The last job run will be at the top. If done fast enough the job should still be running and the status can be viewed real time by clicking on the title of the workflow, which is the commit message supplied before the push, and then clicking on the job title. If all the steps above were followed correctly the job should complete without issue and a new image will have been pushed to the Docker Hub account configured in the repository secrets. Navigate to Docker Hub to check the container registry for the new image. The image can be pulled and run directly from Docker Hub now with something like the following command, just update the image name to match the custom value used in the workflow: 

`docker run -p 5000:5000 ncote/docker-example:2024-02-13.16`

When it starts navigate a browser to [http://127.0.0.1:5000/](http://127.0.0.1:5000/) and the website will be accessible. The new header can be seen and the next steps are to incorporate the image in to a Helm chart that can be tied in to the CISL on-premise cloud continuous deployment tool.

## Create a Helm Chart

There are a few different Helm chart templates that are supplied to help get the ball rolling. Links to each one can be found below:

- [Standalone Web Application](https://github.com/NicholasCote/web-app-helm)
- [Containers with Auto-scaling](https://github.com/NicholasCote/web-app-helm-auto-scale)
- [Web Application with a dedicated Dask Cluster](https://github.com/NicholasCote/web-app-dask-helm)
- [Web Application with a GLADE mount](https://github.com/NicholasCote/web-app-vols-helm)

The application used in this repository is a Standalone Web Application. The files used in that repository can be copied in to this repository and customized for this unique application. The easiest way to integrate the new files is to clone the repository to a new local directory and copy the `web-app-helm/` folder in to this repository.  

```
git clone https://github.com/NicholasCote/web-app-helm.git
cp -r web-app-helm/web-app-helm/ docker-how-to/
```

That folder is now integrated with the docker-how-to repository and changes can be made to customize the Chart.

### Update values.yaml

```{note}
The example uses ncote as the sso user name. Please replace ncote with your own sso user name
```

Inside the `web-app-helm/` directory is a file named `values.yaml` where all the variables needed for the application are defined. For the application being deployed there are 7 lines that need to be updated with custom values. Line 4 is the application name and should be short but descriptive. For this demonstration example using your sso username and how-to like `ncote-how-to` is sufficient. Line 5 is a group name and for this it's fine to use the application name. Line 6 is the applications URL path. For flask the base URL path is just `/` and that is used in this instance. Line 8 is the FQDN we want to use for the applications URL. This is typically just the app name followed by `.k8s.ucar.edu`. Based off the previous app name example the FQDN would be `ncote-how-to.k8s.ucar.edu`. Line 9 is a Kubernetes secret that gets created to store the TLS certificate tied to the FQDN for the application. This needs to be unique for the FQDN and an applicable value would end with the application name. Going with the same example app name the secretName would be `incommon-cert-ncote-how-to`. Line 11 is the container image to use and it should match the image above, for example `ncote/docker-example:2024-02-13.16` was the image created. Line 12 is the port exposed by the container that we want to connect to. For flask the default is 5000 and that is what should be used. The final values.yaml file should look something like this just with ncote being replace with your sso. 

```yaml
replicaCount: 1

webapp:
  name: ncote-how-to
  group: ncote-how-to
  path: /
  tls:
    fqdn: ncote-how-to.k8s.ucar.edu
    secretName: incommon-cert-ncote-how-to
  container: 
    image: hub.k8s.ucar.edu/docker/ncote/docker-example:2024-02-13.16
    port: 5000
    memory: 1G
    cpu: 2
```

### Update Chart.yaml

The `Chart.yaml` file only has a few values that need to be updated to make the application unique. Line 2 is the application name, this should be the same as the application name provided in the `values.yaml` file. Line 3 is a description for the application. This should be something short but descriptive for the application. For this example the following would be applicable `ncote's flask application for how-to tutorial`. Line 18 is the Semantic version and it should be incremented by one every time there is an update to the application. Line 24 is the application version and typically matches the container image tag.   

## Connect to Argo CD

```{note}
This section needs to be done by a CISL on-premise cloud administrator.
```

If you are not following this how-to as part of a CISL on-prem cloud tutorial you will need to get a ticket submitted to get this application added to Argo CD for testing. Since this is just a demonstration it should not be deployed long term and should be deleted shortly after initial deployment. 